In [2]:
%reload_ext autoreload
%autoreload 2

# Imports
Import the required classes and functions.

In [3]:
from mleko.dataset.convert import CSVToVaexConverter
from mleko.dataset.feature_select import (
    MissingRateFeatureSelector,
    VarianceFeatureSelector,
    CompositeFeatureSelector,
    PearsonCorrelationFeatureSelector,
    InvarianceFeatureSelector,
)
from mleko.dataset.ingest import KaggleIngester
from mleko.dataset.split import ExpressionSplitter, RandomSplitter
from mleko.pipeline import Pipeline
from mleko.pipeline.steps import ConvertStep, FeatureSelectStep, IngestStep, SplitStep


# Constants
Define configuration variables.

In [4]:
OWNER_SLUG = 'mlg-ulb'
DATASET_SLUG = 'creditcardfraud'
DATASET_NAME = f'{OWNER_SLUG}/{DATASET_SLUG}'

TARGET_FEATURE = "Class"
TIME_FEATURE = "Time"
META_FEATURES = [TIME_FEATURE, TARGET_FEATURE]
RANDOM_STATE = 1337

# Pipeline Setup

In [5]:
kaggle_data_source = KaggleIngester(
    destination_directory=f"data/{DATASET_NAME}/raw", owner_slug=OWNER_SLUG, dataset_slug=DATASET_SLUG
)
csv_to_arrow_converter = CSVToVaexConverter(
    output_directory=f"data/{DATASET_NAME}/converted", downcast_float=True, random_state=RANDOM_STATE
)
random_data_splitter = RandomSplitter(
    cache_directory=f"data/{DATASET_NAME}/split",
    data_split=(0.80, 0.20),
    shuffle=True,
    stratify=TARGET_FEATURE,
    random_state=RANDOM_STATE,
)
expression_data_splitter = ExpressionSplitter(cache_directory=f"data/{DATASET_NAME}/split", expression="Time > 100")
composite_feature_selector = CompositeFeatureSelector(
    cache_directory=f"data/{DATASET_NAME}/feature_selection",
    feature_selectors=[
        MissingRateFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            missing_rate_threshold=0.7,
            ignore_features=META_FEATURES,
        ),
        VarianceFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            variance_threshold=0.00,
            ignore_features=META_FEATURES
        ),
        PearsonCorrelationFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            correlation_threshold=0.7,
            ignore_features=META_FEATURES
        ),
        InvarianceFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            ignore_features=META_FEATURES
        ),
    ],
)


pipeline = Pipeline(
    steps=[
        IngestStep(kaggle_data_source, outputs=["raw_csv"]),
        ConvertStep(csv_to_arrow_converter, inputs=["raw_csv"], outputs=["df_clean"]),
        SplitStep(random_data_splitter, inputs=["df_clean"], outputs=["df_train_validate", "df_test"]),
        FeatureSelectStep(
            composite_feature_selector,
            inputs=["df_train_validate"],
            outputs=["df_train_validate_features_selected"],
        ),
        SplitStep(expression_data_splitter, inputs=["df_train_validate_features_selected"], outputs=["df_train", "df_validate"]),
    ]
)


[2023-06-03 22:59:19] [INFO] Attempting to fetch Kaggle API credentials from environment variables 'KAGGLE_USERNAME' and 'KAGGLE_KEY'. (kaggle_ingester.py:74)
[2023-06-03 22:59:19] [WARNING] Kaggle API credentials not found in environment variables, attempting to fetch from fallback path at ~/.kaggle/kaggle.json. (kaggle_ingester.py:82)
[2023-06-03 22:59:19] [INFO] Kaggle credentials successfully fetched. (kaggle_ingester.py:91)


# Run Pipeline

In [6]:
data_container =  pipeline.run(force_recompute=True).data

[2023-06-03 22:59:20] [INFO] No data container provided. Creating an empty one. (pipeline.py:77)
[2023-06-03 22:59:20] [INFO] Executing step 1/5: IngestStep. (pipeline.py:81)
[2023-06-03 22:59:21] [INFO] Force Cache Refresh: Downloading mlg-ulb/creditcardfraud/* to data/mlg-ulb/creditcardfraud/raw from Kaggle. (kaggle_ingester.py:287)


[2023-06-03 22:59:26] [INFO] Finished downloading 1 files from Kaggle. (kaggle_ingester.py:303)
[2023-06-03 22:59:26] [INFO] Finished step 1/5 execution. (pipeline.py:83)
[2023-06-03 22:59:26] [INFO] Executing step 2/5: ConvertStep. (pipeline.py:81)


[2023-06-03 22:59:26] [INFO] Force Cache Refresh (LRUCache) CSVToVaexConverter.convert: Executing method. (cache_mixin.py:133)


Writing DataFrame to .arrow file: 100%|██████████| 100/100 [00:00<00:00, 739.42it/s]

[2023-06-03 22:59:34] [INFO] Finished step 2/5 execution. (pipeline.py:83)
[2023-06-03 22:59:34] [INFO] Executing step 3/5: SplitStep. (pipeline.py:81)
[2023-06-03 22:59:35] [INFO] Force Cache Refresh (LRUCache) RandomSplitter.split: Executing method. (cache_mixin.py:133)
[2023-06-03 22:59:35] [INFO] Shuffling data before splitting. (random_splitter.py:122)
[2023-06-03 22:59:35] [INFO] Splitting data with stratification on column 'Class'. (random_splitter.py:126)


...

[2023-06-03 22:59:38] [INFO] Dropping (0) invariant features: set(). (invariance_feature_selector.py:106)
[2023-06-03 22:59:38] [INFO] Finished composite feature selection step 4/4. (composite_feature_selector.py:113)



Writing DataFrame to .arrow file: 100%|██████████| 100/100 [00:00<00:00, 518.01it/s]


[2023-06-03 22:59:38] [INFO] Finished step 4/5 execution. (pipeline.py:83)
[2023-06-03 22:59:38] [INFO] Executing step 5/5: SplitStep. (pipeline.py:81)
[2023-06-03 22:59:38] [INFO] Force Cache Refresh (LRUCache) ExpressionSplitter.split: Executing method. (cache_mixin.py:133)
[2023-06-03 22:59:38] [INFO] Splitting dataframe based on expression 'Time > 100'. (expression_splitter.py:92)
[2023-06-03 22:59:38] [INFO] Split dataframe into two dataframes with shapes (227718, 31) and (127, 31). (expression_splitter.py:95)


Writing DataFrame to .arrow file: 100%|██████████| 100/100 [00:00<00:00, 2517.70it/s]

[2023-06-03 22:59:38] [INFO] Finished step 5/5 execution. (pipeline.py:83)
